Match the APOGEE-red-clump catalog to Gaia DR2, integrate the orbits using galpy for 20 Gyr, and make a density plot in Galactocentric X and Y coordinates of all of the positions of the stars over the next 20 Gyr.

In [11]:
import astropy.units as u
import gaia_tools.load as gload
import gaia_tools.xmatch as gmatch
from galpy.potential import MWPotential2014
from galpy.orbit import Orbit
from astropy.coordinates import SkyCoord
import numpy as np

Load the data from APOGEE red clump and cross match with gaia2

In [12]:
aprc= gload.apogeerc()
gaia2_matches, matches_indx = gmatch.cds(aprc, xcat='vizier:I/345/gaia2')

C:\Users\user\AppData\Roaming\Python\Python36\site-packages\gaia_tools-0.2-py3.6.egg\gaia_tools\load\__init__.py:94: UserWarning: Falling back on simple APOGEE interface; for more functionality, install the jobovy/apogee package
  warnings.warn("Falling back on simple APOGEE interface; for more functionality, install the jobovy/apogee package")



Get the list of the 6 coordinates

In [13]:
# take out all the stars whose radial velocity is -9999.99
gaia2_matches = gaia2_matches[gaia2_matches['radial_velocity'] != -9999.99]

In [14]:
ra_list = gaia2_matches['ra']
dec_list = gaia2_matches['dec']
parallax_list = gaia2_matches['parallax']
vr_list = gaia2_matches['radial_velocity']
pm_ra_list = gaia2_matches['pmra']
pm_dec_list = gaia2_matches['pmdec']

In [15]:
# create time array
ts = np.linspace(0,20,1000)
# initialize orbit list
o = []
# initialize star location plot
star_coord = []

Create an orbit object for each star

In [ ]:
%pylab inline
import time
start= time.time()
for i in range(np.size(gaia2_matches)):
    ra, dec, parallax, vr, v_ra, v_dec = ra_list[i], dec_list[i], parallax_list[i], vr_list[i], pm_ra_list[i], pm_dec_list[i]
    # use parallex to find radial distance
    d = 1/parallax # since parallax is given in milliarcsecond, its inverse is kpc
    # create orbit object
    o.append(Orbit(vxvv = [ra,dec,d,v_ra, v_dec,vr], radec = True))
    o[i].integrate(ts,MWPotential2014)
    # store the x,y coordinate of each star after 20 Gyr
    coordinate = (o[i].x(20),o[i].y(20))
    star_coord.append(coordinate)
    if i%100 == 0:
        time_now = time.time() - start
        print(i/np.size(gaia2_matches), ", ", time_now)
    
    
# get the x,y coordinate into an array and save it
star_coord = np.array(star_coord)
x = star_coord[:, 0]
y = star_coord[:,1]
np.savez("x_y.npz", x = x, y = y)

Populating the interactive namespace from numpy and matplotlib
0.0 ,  0.1520085334777832
0.008121497604158206 ,  10.204583644866943
0.016242995208316412 ,  19.773130893707275
0.02436449281247462 ,  29.335678100585938
0.032485990416632825 ,  37.87516641616821
0.04060748802079103 ,  46.29864835739136
0.04872898562494924 ,  55.14515423774719
0.05685048322910745 ,  65.09472322463989
0.06497198083326565 ,  74.07323694229126
0.07309347843742386 ,  83.22376036643982
0.08121497604158207 ,  92.54029297828674
0.08933647364574028 ,  102.00783467292786
0.09745797124989848 ,  111.66638708114624
0.10557946885405668 ,  121.40694427490234
0.1137009664582149 ,  131.95154738426208
0.1218224640623731 ,  142.44214725494385
0.1299439616665313 ,  151.84668517112732
0.13806545927068953 ,  162.10727214813232
0.14618695687484773 ,  172.32685661315918
0.15430845447900593 ,  181.7943980693817
0.16242995208316413 ,  194.02909803390503
0.17055144968732233 ,  203.66864919662476
0.17867294729148056 ,  213.9312362670

In [ ]:
import pylab as plt
plt.scatter(x,y, s = 0.1)
plt.xlim(-30,30)
plt.ylim(-30,30)
plt.savefig("20 Gyr density plot.pdf")

In [26]:
data = np.load("x_y.npz")
nx=data['x']
ny=data['y']

print(x==nx)

[ True  True  True ...  True  True  True]
